# Lesson 3: Reflection and Blogpost Writing

## Setup

In [1]:
llm_config = {"model": "gpt-3.5-turbo"}

## The task!

In [2]:
task = '''
        Write a concise but engaging blogpost about
       DeepLearning.AI. Make sure the blogpost is
       within 100 words.
       '''


## Create a writer agent

In [3]:
import autogen

writer = autogen.AssistantAgent(
    name="Writer",
    system_message="You are a writer. You write engaging and concise " 
        "blogpost (with title) on given topics. You must polish your "
        "writing based on the feedback you receive and give a refined "
        "version. Only return your final work without additional comments.",
    llm_config=llm_config,
)

In [4]:
reply = writer.generate_reply(messages=[{"content": task, "role": "user"}])

In [5]:
print(reply)

Title: Unveiling the Power of DeepLearning.AI

Dive into the world of cutting-edge technology with DeepLearning.AI—a premier platform redefining artificial intelligence. Led by industry expert Andrew Ng, it offers top-tier courses on deep learning, neural networks, and more. Whether you're a beginner or a seasoned professional, DeepLearning.AI equips you with the knowledge and skills to thrive in this rapidly evolving field. Join thousands of learners worldwide in unraveling the magic of AI. Elevate your career and unleash your potential with DeepLearning.AI today!


## Adding reflection 

Create a critic agent to reflect on the work of the writer agent.

In [6]:
critic = autogen.AssistantAgent(
    name="Critic",
    is_termination_msg=lambda x: x.get("content", "").find("TERMINATE") >= 0,
    llm_config=llm_config,
    system_message="You are a critic. You review the work of "
                "the writer and provide constructive "
                "feedback to help improve the quality of the content.",
)

In [7]:
res = critic.initiate_chat(
    recipient=writer,
    message=task,
    max_turns=2,
    summary_method="last_msg"
)

Critic (to Writer):


        Write a concise but engaging blogpost about
       DeepLearning.AI. Make sure the blogpost is
       within 100 words.
       

--------------------------------------------------------------------------------
Writer (to Critic):

Title: Unveiling the Power of DeepLearning.AI

Dive into the world of cutting-edge technology with DeepLearning.AI—a premier platform redefining artificial intelligence. Led by industry expert Andrew Ng, it offers top-tier courses on deep learning, neural networks, and more. Whether you're a beginner or a seasoned professional, DeepLearning.AI equips you with the knowledge and skills to thrive in this rapidly evolving field. Join thousands of learners worldwide in unraveling the magic of AI. Elevate your career and unleash your potential with DeepLearning.AI today!

--------------------------------------------------------------------------------
Critic (to Writer):

This blogpost does a great job of summarizing the key features an

## Nested chat

In [8]:
SEO_reviewer = autogen.AssistantAgent(
    name="SEO Reviewer",
    llm_config=llm_config,
    system_message="You are an SEO reviewer, known for "
        "your ability to optimize content for search engines, "
        "ensuring that it ranks well and attracts organic traffic. " 
        "Make sure your suggestion is concise (within 3 bullet points), "
        "concrete and to the point. "
        "Begin the review by stating your role.",
)


In [9]:
legal_reviewer = autogen.AssistantAgent(
    name="Legal Reviewer",
    llm_config=llm_config,
    system_message="You are a legal reviewer, known for "
        "your ability to ensure that content is legally compliant "
        "and free from any potential legal issues. "
        "Make sure your suggestion is concise (within 3 bullet points), "
        "concrete and to the point. "
        "Begin the review by stating your role.",
)

In [10]:
ethics_reviewer = autogen.AssistantAgent(
    name="Ethics Reviewer",
    llm_config=llm_config,
    system_message="You are an ethics reviewer, known for "
        "your ability to ensure that content is ethically sound "
        "and free from any potential ethical issues. " 
        "Make sure your suggestion is concise (within 3 bullet points), "
        "concrete and to the point. "
        "Begin the review by stating your role. ",
)

In [11]:
meta_reviewer = autogen.AssistantAgent(
    name="Meta Reviewer",
    llm_config=llm_config,
    system_message="You are a meta reviewer, you aggragate and review "
    "the work of other reviewers and give a final suggestion on the content.",
)

## Orchestrate the nested chats to solve the task

In [13]:
def reflection_message(recipient, messages, sender, config):
    return f'''Review the following content. 
            \n\n {recipient.chat_messages_for_summary(sender)[-1]['content']}'''

review_chats = [
    {
     "recipient": SEO_reviewer, 
     "message": reflection_message, 
     "summary_method": "reflection_with_llm",
     "summary_args": {"summary_prompt" : 
        "Return review into as JSON object only:"
        "{'Reviewer': '', 'Review': ''}. Here Reviewer should be your role",},
     "max_turns": 1},
    {
    "recipient": legal_reviewer, "message": reflection_message, 
     "summary_method": "reflection_with_llm",
     "summary_args": {"summary_prompt" : 
        "Return review into as JSON object only:"
        "{'Reviewer': '', 'Review': ''}.",},
     "max_turns": 1},
    {"recipient": ethics_reviewer, "message": reflection_message, 
     "summary_method": "reflection_with_llm",
     "summary_args": {"summary_prompt" : 
        "Return review into as JSON object only:"
        "{'reviewer': '', 'review': ''}",},
     "max_turns": 1},
     {"recipient": meta_reviewer, 
      "message": "Aggregrate feedback from all reviewers and give final suggestions on the writing.", 
     "max_turns": 1},
]


In [14]:
critic.register_nested_chats(
    review_chats,
    trigger=writer,
)

**Note**: You might get a slightly different response than what's shown in the video. Feel free to try different task.

In [15]:
res = critic.initiate_chat(
    recipient=writer,
    message=task,
    max_turns=2,
    summary_method="last_msg"
)

Critic (to Writer):


        Write a concise but engaging blogpost about
       DeepLearning.AI. Make sure the blogpost is
       within 100 words.
       

--------------------------------------------------------------------------------
Writer (to Critic):

Title: Unveiling the Power of DeepLearning.AI

Dive into the world of cutting-edge technology with DeepLearning.AI—a premier platform redefining artificial intelligence. Led by industry expert Andrew Ng, it offers top-tier courses on deep learning, neural networks, and more. Whether you're a beginner or a seasoned professional, DeepLearning.AI equips you with the knowledge and skills to thrive in this rapidly evolving field. Join thousands of learners worldwide in unraveling the magic of AI. Elevate your career and unleash your potential with DeepLearning.AI today!

--------------------------------------------------------------------------------

********************************************************************************
Star


--------------------------------------------------------------------------------
Critic (to Writer):

Aggregated Feedback:
1. SEO Reviewer suggests improving the title for better search optimization by strategically including relevant keywords and optimizing the meta description.
2. Legal Reviewer advises ensuring the usage of industry-specific terms and trademarks complies with intellectual property regulations, verifying the accuracy of claims, and reviewing guarantees or promises to prevent misrepresentation.

Final Suggestions:
1. Enhance the title by incorporating relevant keywords to boost search visibility.
2. Ensure compliance with intellectual property regulations when using industry-specific terms and trademarks related to artificial intelligence.
3. Verify the accuracy of any claims made about the platform and courses offered.
4. Review guarantees or promises about career advancement to prevent potential misrepresentation.
5. Strive for a balance between SEO optimization an

## Get the summary

In [16]:
print(res.summary)

Title: Master Artificial Intelligence with DeepLearning.AI

Embark on a journey of discovery with DeepLearning.AI—an authoritative platform revolutionizing artificial intelligence education. Helmed by renowned expert Andrew Ng, it delivers unparalleled courses on deep learning and neural networks. Whether you're a novice or a seasoned pro, DeepLearning.AI empowers you with the expertise to excel in this dynamic domain. Join a global community of learners unraveling the wonders of AI. Ignite your career and realize your potential with DeepLearning.AI today!
